In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime

In [2]:
from deep_reinforcement_learning.environment.prediction.transform_input_data import (
    build_features_residualized,
    build_targets_residualized,
    pick_cols,
    make_time_features,
    InputDataBuilderDP,
)
from deep_reinforcement_learning.environment.prediction.model import (
    load_residual_model,
    predict_full_period_with_residual_model,
)

In [3]:
from datetime import datetime

import pandas as pd
from prettytable import PrettyTable


class DataManager:
    @classmethod
    def show_status(self, df: pd.DataFrame):
        table = PrettyTable(
            ["Variable", "Type", "Missing Values", "Duplicates", "Outliers"]
        )

        for col in df.columns:
            # データ型の判定
            if pd.api.types.is_numeric_dtype(df[col]):
                col_type = "Numerical"
            else:
                col_type = "Categorical"

            # 欠損値
            missing_values = df[col].isnull().sum()

            # 重複数
            duplicates = df.duplicated(subset=[col]).sum()

            # 外れ値（数値型のみ）
            if col_type == "Numerical":
                mean = df[col].mean()
                std = df[col].std()
                outliers = ((df[col] - mean).abs() > 3 * std).sum()
            else:
                outliers = "N/A"

            table.add_row([col, col_type, missing_values, duplicates, outliers])
        print(table)

    @classmethod
    def extract_term_data(
        cls, target_col: str, df: pd.DataFrame, start_time: datetime, end_time: datetime
    ):
        df[target_col] = pd.to_datetime(df[target_col])
        filtered_df = df[(df[target_col] >= start_time) & (df[target_col] <= end_time)]
        return filtered_df

    @classmethod
    def check_date_sequence(cls, df: pd.DataFrame, date_col: str = "date"):
        dates = pd.to_datetime(df[date_col]).sort_values().reset_index(drop=True)
        expected = pd.date_range(start=dates.min(), end=dates.max(), freq="D")
        missing = expected.difference(dates)
        return {
            "is_continuous": len(missing) == 0,
            "missing_dates": missing.strftime("%Y-%m-%d").tolist(),
        }

In [5]:
import os
os.getcwd()

'c:\\Users\\iamke\\OneDrive\\doc\\MENTERU\\AIrux8_opti_logic'

In [6]:
base_df = pd.read_csv("data/base/hourly_filled.csv")
base_df["Datetime_hour"] = pd.to_datetime(base_df["Datetime_hour"])
base_df = base_df.set_index("Datetime_hour", drop=True)

In [7]:
def split_df(df, term):
    _df = df.copy()
    historical_df = _df[_df.index < term]
    validate_df = _df[_df.index >= term]
    return historical_df, validate_df

In [9]:
from datetime import datetime
import pandas as pd

# ===== 予測したい時刻 =====
t0 = pd.Timestamp("2025-09-10 07:00:00")

# ===== 学習/検証に分割（t0を境に“過去”と“将来”）=====
historical_df, validate_df = split_df(base_df, datetime(2025, 9, 10, 7))

# ===== DPビルダーは“過去のみ”でfit（重要：リーク防止）=====
builder = InputDataBuilderDP(
    days=7, lags_hours=(1,), include_weather_raw=True, include_original_controls=True
)
_ = builder.fit(historical_df)
builder.begin_online(historical_df)
# ===== その時刻の weather/controls を用意（なければフォールバック）=====
weather_cols = ["Outdoor Temp.", "Outdoor Humidity", "Solar Radiation"]

set_cols = [
    "A/C Set Temperature__A-25",
    "A/C Set Temperature__A-26",
    "A/C Set Temperature__D-1南1",
    "A/C Set Temperature__D-2北1",
    "A/C Set Temperature__D-3南2",
    "A/C Set Temperature__D-4北2",
    "A/C Set Temperature__D-5南1",
    "A/C Set Temperature__D-6北1",
    "A/C Set Temperature__D-7南2",
    "A/C Set Temperature__D-8北2",
    "A/C Set Temperature__E-10南2",
    "A/C Set Temperature__E-11南3",
    "A/C Set Temperature__E-12南4",
    "A/C Set Temperature__E-13北1",
    "A/C Set Temperature__E-14北2",
    "A/C Set Temperature__E-15北3",
    "A/C Set Temperature__E-16北4",
    "A/C Set Temperature__E-17",
    "A/C Set Temperature__E-9南1",
    "A/C Set Temperature__F-18",
    "A/C Set Temperature__F-19",
    "A/C Set Temperature__F-20",
    "A/C Set Temperature__G-21",
    "A/C Set Temperature__G-22",
    "A/C Set Temperature__G-23",
    "A/C Set Temperature__G-24",
]
mode_cols = [
    "A/C Mode__A-25",
    "A/C Mode__A-26",
    "A/C Mode__D-1南1",
    "A/C Mode__D-2北1",
    "A/C Mode__D-3南2",
    "A/C Mode__D-4北2",
    "A/C Mode__D-5南1",
    "A/C Mode__D-6北1",
    "A/C Mode__D-7南2",
    "A/C Mode__D-8北2",
    "A/C Mode__E-10南2",
    "A/C Mode__E-11南3",
    "A/C Mode__E-12南4",
    "A/C Mode__E-13北1",
    "A/C Mode__E-14北2",
    "A/C Mode__E-15北3",
    "A/C Mode__E-16北4",
    "A/C Mode__E-17",
    "A/C Mode__E-9南1",
    "A/C Mode__F-18",
    "A/C Mode__F-19",
    "A/C Mode__F-20",
    "A/C Mode__G-21",
    "A/C Mode__G-22",
    "A/C Mode__G-23",
    "A/C Mode__G-24",
]
fan_cols = [
    "A/C Fan Speed__A-25",
    "A/C Fan Speed__A-26",
    "A/C Fan Speed__D-1南1",
    "A/C Fan Speed__D-2北1",
    "A/C Fan Speed__D-3南2",
    "A/C Fan Speed__D-4北2",
    "A/C Fan Speed__D-5南1",
    "A/C Fan Speed__D-6北1",
    "A/C Fan Speed__D-7南2",
    "A/C Fan Speed__D-8北2",
    "A/C Fan Speed__E-10南2",
    "A/C Fan Speed__E-11南3",
    "A/C Fan Speed__E-12南4",
    "A/C Fan Speed__E-13北1",
    "A/C Fan Speed__E-14北2",
    "A/C Fan Speed__E-15北3",
    "A/C Fan Speed__E-16北4",
    "A/C Fan Speed__E-17",
    "A/C Fan Speed__E-9南1",
    "A/C Fan Speed__F-18",
    "A/C Fan Speed__F-19",
    "A/C Fan Speed__F-20",
    "A/C Fan Speed__G-21",
    "A/C Fan Speed__G-22",
    "A/C Fan Speed__G-23",
    "A/C Fan Speed__G-24",
]
onoff_cols = [
    "A/C ON/OFF__A-25",
    "A/C ON/OFF__A-26",
    "A/C ON/OFF__D-1南1",
    "A/C ON/OFF__D-2北1",
    "A/C ON/OFF__D-3南2",
    "A/C ON/OFF__D-4北2",
    "A/C ON/OFF__D-5南1",
    "A/C ON/OFF__D-6北1",
    "A/C ON/OFF__D-7南2",
    "A/C ON/OFF__D-8北2",
    "A/C ON/OFF__E-10南2",
    "A/C ON/OFF__E-11南3",
    "A/C ON/OFF__E-12南4",
    "A/C ON/OFF__E-13北1",
    "A/C ON/OFF__E-14北2",
    "A/C ON/OFF__E-15北3",
    "A/C ON/OFF__E-16北4",
    "A/C ON/OFF__E-17",
    "A/C ON/OFF__E-9南1",
    "A/C ON/OFF__F-18",
    "A/C ON/OFF__F-19",
    "A/C ON/OFF__F-20",
    "A/C ON/OFF__G-21",
    "A/C ON/OFF__G-22",
    "A/C ON/OFF__G-23",
    "A/C ON/OFF__G-24",
]
all_control_cols = set_cols + mode_cols + fan_cols + onoff_cols


# t0の行を最優先に取得。無ければ「直近過去で前方埋め」フォールバック。
def _row_at_t0_or_ffill(df, cols, t0):
    cols = [c for c in cols if c in df.columns]
    if t0 in df.index:
        return df.loc[[t0], cols]
    # 直近過去1行で埋める
    return df.loc[:t0, cols].tail(1).reindex(columns=cols)


# 入力3点セット（1行だけ）
time_info = pd.DataFrame(index=pd.DatetimeIndex([t0]))
weather_df = _row_at_t0_or_ffill(validate_df, weather_cols, t0).copy()
control_df = _row_at_t0_or_ffill(validate_df, all_control_cols, t0).copy()

# ===== Xを1行だけ生成（DP内部でBL/ラグは historical_df に基づく）=====
X = builder.make_input_next(t0, weather_df, control_df)

# ===== モデル読込＆列順を学習時に強制整列 =====
model = load_residual_model("models/xgb_weight.joblib")

# ===== 予測（残差→元スケールへ加算）=====
res_all = predict_full_period_with_residual_model(
    model=model,
    X_full=X,
    model_target_names=list(model.y_cols),
    wanted_target_cols=list(model.y_cols),  # 全ターゲット返す。必要なら部分列に変更
    bl_prefix="bl__",
    add_back_baseline=True,
)
y_7 = res_all["y_pred"]

In [10]:
control_df

,A/C Set Temperature__A-25,A/C Set Temperature__A-26,A/C Set Temperature__D-1南1,A/C Set Temperature__D-2北1,A/C Set Temperature__D-3南2,A/C Set Temperature__D-4北2,A/C Set Temperature__D-5南1,A/C Set Temperature__D-6北1,A/C Set Temperature__D-7南2,A/C Set Temperature__D-8北2,...,A/C ON/OFF__E-16北4,A/C ON/OFF__E-17,A/C ON/OFF__E-9南1,A/C ON/OFF__F-18,A/C ON/OFF__F-19,A/C ON/OFF__F-20,A/C ON/OFF__G-21,A/C ON/OFF__G-22,A/C ON/OFF__G-23,A/C ON/OFF__G-24
Datetime_hour,,,,,,,,,,,,,,,,,,,,,
2025-09-10 07:00:00,18.0,26.0,21.0,24.0,23.0,24.0,24.0,24.0,24.0,24.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# builder.accumulate_actuals(y_7)

In [11]:
DataManager.show_status(y_7)

+----------------------+-----------+----------------+------------+----------+
|       Variable       |    Type   | Missing Values | Duplicates | Outliers |
+----------------------+-----------+----------------+------------+----------+
|   total_kwh__41-1    | Numerical |       0        |     0      |    0     |
|   total_kwh__43-1    | Numerical |       0        |     0      |    0     |
|   total_kwh__43-2    | Numerical |       0        |     0      |    0     |
|   total_kwh__43-3    | Numerical |       0        |     0      |    0     |
|   total_kwh__43-4    | Numerical |       0        |     0      |    0     |
|   total_kwh__44-1    | Numerical |       0        |     0      |    0     |
|   total_kwh__44-2    | Numerical |       0        |     0      |    0     |
|   total_kwh__44-3    | Numerical |       0        |     0      |    0     |
|   total_kwh__44-4    | Numerical |       0        |     0      |    0     |
|   total_kwh__44-5    | Numerical |       0        |     0     

In [12]:
weather_df

,Outdoor Temp.,Outdoor Humidity,Solar Radiation
Datetime_hour,,,
2025-09-10 07:00:00,28.5,80.0,82.4


In [13]:
time_info

""
2025-09-10 07:00:00


In [14]:
y_7["total_kwh__49-9"]

2025-09-10 07:00:00    3282.05828
Name: total_kwh__49-9, dtype: float64

In [15]:
historical_df

,A/C Set Temperature__A-25,A/C Set Temperature__A-26,A/C Set Temperature__D-1南1,A/C Set Temperature__D-2北1,A/C Set Temperature__D-3南2,A/C Set Temperature__D-4北2,A/C Set Temperature__D-5南1,A/C Set Temperature__D-6北1,A/C Set Temperature__D-7南2,A/C Set Temperature__D-8北2,...,total_kwh__49-3,total_kwh__49-4,total_kwh__49-6,total_kwh__49-7,total_kwh__49-8,total_kwh__49-9,datetime,Outdoor Temp.,Outdoor Humidity,Solar Radiation
Datetime_hour,,,,,,,,,,,,,,,,,,,,,
2024-06-30 15:00:00,24.0,25.0,27.0,27.000000,27.0,27.000000,27.0,27.0,27.0,27.0,...,325.0,420.0,520.0,695.0,610.0,770.0,2024-06-30 15:00:00,27.3,77.88,420.7
2024-06-30 16:00:00,24.0,25.0,27.0,27.000000,27.0,27.000000,27.0,27.0,27.0,27.0,...,260.0,240.0,455.0,700.0,575.0,1050.0,2024-06-30 16:00:00,27.1,79.76,188.7
2024-06-30 17:00:00,24.0,25.0,27.0,27.000000,27.0,27.000000,27.0,27.0,27.0,27.0,...,65.0,185.0,455.0,420.0,380.0,1820.0,2024-06-30 17:00:00,26.5,81.63,109.5
2024-06-30 18:00:00,24.0,25.0,27.0,27.000000,27.0,27.000000,27.0,27.0,27.0,27.0,...,325.0,480.0,130.0,400.0,605.0,1260.0,2024-06-30 18:00:00,26.5,81.14,34.6
2024-06-30 19:00:00,24.0,25.0,27.0,27.000000,27.0,27.000000,27.0,27.0,27.0,27.0,...,390.0,360.0,260.0,485.0,480.0,1190.0,2024-06-30 19:00:00,25.2,91.41,8.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-10 02:00:00,18.0,26.0,21.0,23.000000,23.0,23.750000,24.0,24.0,24.0,24.0,...,360.0,14780.0,20570.0,9805.0,10340.0,5780.0,2025-09-10 02:00:00,27.5,85.63,0.0
2025-09-10 03:00:00,18.0,26.0,21.0,23.250000,23.0,23.333333,24.0,24.0,24.0,24.0,...,420.0,26715.0,9590.0,17530.0,8935.0,7690.0,2025-09-10 03:00:00,27.5,85.32,0.0
2025-09-10 04:00:00,18.0,26.0,21.0,24.000000,23.0,24.000000,24.0,24.0,24.0,24.0,...,240.0,21940.0,24670.0,15465.0,9940.0,8230.0,2025-09-10 04:00:00,27.5,85.62,0.0


In [13]:
time_info

""
2025-09-10 07:00:00
